In [4]:
import pygame
from pygame.locals import *
import time
import random
import sys

pygame.init()
clock = pygame.time.Clock()
frame_per_sec = 50
screen_width = 735
screen_height = 640
screen = pygame.display.set_mode((screen_width, screen_height))
pygame.display.set_caption('Dragon Dash')

# game variables
ground_scroll = 0
scroll_speed = 4
flying = False
game_over = False
game_started = False

slow_frequency = 20000
dash_frequency = 15000
waiting = True
last_slow = pygame.time.get_ticks() - slow_frequency
last_dash = pygame.time.get_ticks() - dash_frequency

dash_active = False
dash_start_time = 0
normal_scroll_speed = 4
dash_scroll_speed = 12
speed_lines = []

icon_delay = 5  # 5 seconds delay

score = 0
bonus_score = 0
game_start_time = 0
game_end_time = 0
font = pygame.font.Font(None, 36)

background = pygame.image.load("bgd (1) (1).png")
ground = pygame.image.load("ground (4).png")

# setting up the dragon
class Dragon(pygame.sprite.Sprite):
    def __init__(self, x, y):
        pygame.sprite.Sprite.__init__(self)
        self.images = []
        self.index = 0
        self.counter = 0
        for num in range(1,4):
            img = pygame.image.load(f"dr{num}.png")
            self.images.append(img)
        self.image = self.images[self.index]
        self.rect = self.image.get_rect()
        self.rect.center = [x, y]
        self.vel = 0
        self.jumped = False
        self.dashing = False

    def update(self):
        global dash_active, dash_start_time
        
        if flying == True and not self.dashing:
            # apply gravity only when not dashing 
            self.vel += 0.5
            if self.vel > 8:
                self.vel = 8
            if self.rect.bottom < 600:
                self.rect.y += int(self.vel)

        if game_over == False:
            # jump
            keys = pygame.key.get_pressed()
            if keys[pygame.K_UP] and not self.jumped:
                self.jumped = True
                self.vel = -10
            if not keys[pygame.K_UP]:
                self.jumped = False

            # dash
            self.dashing = dash_active and keys[pygame.K_RIGHT]
            if self.dashing:
                # reset vertical velocity when dashing
                self.vel = 0

            # animation of the dragon
            self.counter += 1
            flap_cooldown = 5
            if self.counter > flap_cooldown:
                self.counter = 0
                self.index += 1
            if self.index >= len(self.images):
                self.index = 0
            self.image = self.images[self.index]
            self.image = pygame.transform.rotate(self.images[self.index], self.vel * -1)
        
        else:
            self.image = pygame.transform.rotate(self.images[self.index], -90)

class Spike(pygame.sprite.Sprite):
    def __init__(self, x, y):
        pygame.sprite.Sprite.__init__(self)
        self.image = pygame.image.load("spikes1.png")
        self.rect = self.image.get_rect()
        self.rect.topleft = [x, y]
        self.mask = pygame.mask.from_surface(self.image)

    def update(self):
        if not game_over:
            self.rect.x -= scroll_speed
        if self.rect.right < 0:
            self.kill()

class Slow(pygame.sprite.Sprite):
    def __init__(self, x,y):
        pygame.sprite.Sprite.__init__(self)
        self.image = pygame.image.load('slowdown_icon.png')
        self.rect = self.image.get_rect()
        self.rect.topleft = [x,y]
        
    def update(self):
        if flying:
            self.rect.x -= scroll_speed
            if self.rect.right<0:
                self.kill()

class Dash(pygame.sprite.Sprite):
    def __init__(self, x, y):
        pygame.sprite.Sprite.__init__(self)
        self.image = pygame.image.load("dash_icon.png")
        self.rect = self.image.get_rect()
        self.rect.topleft = [x, y]
    
    def update(self):
        if flying:
            self.rect.x -= scroll_speed
            if self.rect.right < 0:
                self.kill()
    
def trigger_delay():
    pygame.time.set_timer(pygame.USEREVENT, 3000)

def create_speed_lines():
    for _ in range(10):
        x = random.randint(screen_width, screen_width + 100)
        y = random.randint(0, screen_height)
        length = random.randint(20, 50)
        speed_lines.append([x, y, length])

def update_speed_lines():
    if not game_over:
        for line in speed_lines:
            line[0] -= 15  # Move faster than scroll speed
            if line[0] < -line[2]:
                line[0] = random.randint(screen_width, screen_width + 100)
                line[1] = random.randint(0, screen_height)

def draw_speed_lines(screen):
    for x, y, length in speed_lines:
        pygame.draw.line(screen, (255, 255, 255), (x, y), (x + length, y), 2)

def screen_shake():
    if not game_over:
        original_x = 0
        shake_amount = 5
        screen.blit(background, (original_x + random.randint(-shake_amount, shake_amount), 0))
    else:
        screen.blit(background, (0, 0))

def reset_game():
    global flying, game_over, game_started, ground_scroll, scroll_speed, dash_active, score, game_start_time, bonus_score
    flying = False
    game_over = False
    game_started = False
    ground_scroll = 0
    scroll_speed = 4
    dash_active = False
    flappy.rect.center = [100, int(screen_height / 2)]
    flappy.vel = 0
    score = 0
    bonus_score = 0 
    game_start_time = time.time()
    spike_group.empty()
    slow_group.empty()
    dash_group.empty()

dragon_group = pygame.sprite.Group()
spike_group = pygame.sprite.Group()
slow_group = pygame.sprite.Group()
dash_group = pygame.sprite.Group()

flappy = Dragon(100, int(screen_height / 2))
dragon_group.add(flappy)

create_speed_lines()

generate_spike = pygame.USEREVENT + 1
pygame.time.set_timer(generate_spike, 1500)

run = True
while run:
    clock.tick(frame_per_sec)
        
    if dash_active and pygame.key.get_pressed()[pygame.K_RIGHT]:
        screen_shake()
    else:
        screen.blit(background, (0,0))
    
    screen.blit(ground, (ground_scroll, 600))
    
    dragon_group.update()
    dragon_group.draw(screen)
    spike_group.draw(screen)
    spike_group.update()
    slow_group.draw(screen)
    slow_group.update()
    dash_group.draw(screen)
    dash_group.update()

    if dash_active and pygame.key.get_pressed()[pygame.K_RIGHT] and not game_over:
        update_speed_lines()
        draw_speed_lines(screen)

    if pygame.sprite.groupcollide(dragon_group, spike_group, False, False, pygame.sprite.collide_mask) or flappy.rect.top < 0:
        if dash_active and pygame.key.get_pressed()[pygame.K_RIGHT]:
            collided_spikes = pygame.sprite.groupcollide(dragon_group, spike_group, False, True, pygame.sprite.collide_mask)
            if collided_spikes:
                bonus_score += 10 * len(collided_spikes[flappy])
        else:
            game_over = True
            game_end_time = time.time()

    if game_started and not game_over:
        current_time = time.time()
        time_score = int(current_time - game_start_time)
        total_score = time_score + bonus_score
        score_text = font.render(f"Score: {total_score}", True, (255, 255, 255))
        screen.blit(score_text, (screen_width - 150, 10))

        time_now = pygame.time.get_ticks()
        if time_now - last_slow > slow_frequency + random.randint(2000, 8000):
            slow_height = random.randint(-200, 200)
            slow = Slow(screen_width, int(screen_height / 2) + slow_height)
            slow_group.add(slow)
            last_slow = time_now

        if time_now - last_dash > dash_frequency + random.randint(2000, 8000):
            dash_height = random.randint(-200, 200)
            dash = Dash(screen_width, int(screen_height / 2) + dash_height)
            dash_group.add(dash)
            last_dash = time_now

        collided_dash = pygame.sprite.spritecollide(flappy, dash_group, True)
        if collided_dash:
            dash_active = True
            dash_start_time = current_time

        if dash_active and current_time - dash_start_time > 7:
            dash_active = False
    
    if pygame.sprite.groupcollide(dragon_group, slow_group, False, True):
        normal_scroll_speed -= 1.5
        start_time = pygame.time.get_ticks()
    if 'start_time' in locals():
        current_time = pygame.time.get_ticks()
        elapsed_time = current_time - start_time
        if elapsed_time >= 2000:
            normal_scroll_speed += 1.5
            del start_time
            
    if flappy.rect.bottom > 600:
        game_over = True
        flying = False

    if game_over == False:
        if dash_active and pygame.key.get_pressed()[pygame.K_RIGHT]:
            scroll_speed = dash_scroll_speed
        else:
            scroll_speed = normal_scroll_speed

        if game_started:
            ground_scroll -= scroll_speed
            if abs(ground_scroll) > 55:
                ground_scroll = 0
    
    for event in pygame.event.get():
        if event.type == pygame.QUIT:
            run = False
        if event.type == pygame.KEYDOWN:
            if event.key == pygame.K_UP:
                if not game_started:
                    game_started = True
                    flying = True
                    game_start_time = time.time()
                elif not flying and not game_over:
                    flying = True
        if event.type == generate_spike and game_started and not game_over:
            if not spike_group or min(spike.rect.right for spike in spike_group) < screen_width - 200:
                if random.random() < 0.6:
                    spike_y = random.randint(50, 250)  # top spikes
                    spike = Spike(screen_width, spike_y)
                else:
                    spike_y = random.randint(300, 500)  # bottom spikes
                    spike = Spike(screen_width, spike_y)
                spike_group.add(spike)
        if event.type == pygame.MOUSEBUTTONDOWN:
            if game_over:
                reset_game()

    if not game_started:
        screen.blit(pygame.image.load("control (2).png"), (160.5, 160))
        
    elif game_over:
        game_over_text = font.render("Game Over", True, (255, 255, 255))
        game_over_rect = game_over_text.get_rect(center=(screen_width/2, screen_height/2 - 60))
        screen.blit(game_over_text, game_over_rect)
        
        time_score = max(1, int(current_time - game_start_time))
        total_score = time_score + bonus_score
        
        final_score = font.render(f"Final Score: {total_score}", True, (255, 255, 255))
        final_score_rect = final_score.get_rect(center=(screen_width/2, screen_height/2 - 20))
        screen.blit(final_score, final_score_rect)
        
        restart_text = font.render("Click to Restart", True, (255, 255, 255))
        restart_rect = restart_text.get_rect(center=(screen_width/2, screen_height/2 + 50))
        screen.blit(restart_text, restart_rect)

    pygame.display.update()

pygame.quit()
sys.exit()

SystemExit: 

/opt/anaconda3/lib/python3.11/site-packages/IPython/core/interactiveshell.py:3561: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
